In [1]:
import tensorflow as tf
from BNN import BNN
from preprocess_data import *
import numpy as np
from utils import *
import matplotlib.pyplot as plt
import os

import subprocess

Load data

In [2]:
data_file = 'vmro3_refC1SD_70x36_13mdls_masked_extrap_and_interp.pkl'
X_train, y_train, X_test, y_test, X_interp, y_interp, X_extrap, y_extrap, X_at, y_at = read_data(data_file)

Number of obs: 439203
Training on 83.1%
Testing on 4.4%
Validation (temporal extrapolation) on 6.8%
Validation (interpolation) on 5.7%


Load BNNs

In [5]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

# num_NNs needs changing
num_NNs = 48
checkpoint_dir = './pretrained-checkpoints/model{}/'
log_dir = './logs/model{}/'
x_dim = X_train.shape[1]
y_dim = y_train.shape[1]
num_models = 13
n_data = X_train.shape[0]
hidden_size = 500
learning_rate = 0.0001

# Priors 
bias_mean = 0.00
bias_std = 0.02
noise_mean = 0.03
noise_std = 0.02
layer_scale = 1.1

alpha_dim = x_dim - num_models
init_std_1w =  np.sqrt(150.0/(alpha_dim))
init_std_1b = init_std_1w
init_std_2w =  (layer_scale)/np.sqrt(hidden_size)
init_std_2b = init_std_2w
init_std_biasw = (1.05 * layer_scale * bias_std)/np.sqrt(hidden_size)
init_std_noisew = (2.5)/np.sqrt(hidden_size)
lambda_anchor = 1.0/(np.array([init_std_1w,init_std_1b,init_std_2w,init_std_2b,init_std_biasw,init_std_noisew])**2)

In [6]:
num_gpus = 4
num_ens_members = 48
num_par_pro = 2
num_ens_per_pro = num_ens_members // (num_gpus * num_par_pro)

graphs = np.repeat([tf.Graph() for i in range(num_gpus * num_par_pro)], num_ens_per_pro)

ep_ = 100000

NNs = []
for i in range(num_NNs):
    with graphs[i].as_default():
        print('On graph {}:'.format(i))
        print('')
        sess = tf.Session(graph=graphs[i], config=config)
        NNs.append(BNN(sess, checkpoint_dir.format(i + 1),
                       log_dir.format(i + 1),
                       x_dim, y_dim, num_models, n_data,
                       hidden_size, learning_rate, lambda_anchor, 
                       init_std_1w, init_std_1b, init_std_2w,
                       init_std_2b, init_std_biasw, init_std_noisew,
                       load_from_epoch=ep_))

On graph 0:











Reading checkpoints...
Loading from checkpoint: BNN-100000
./pretrained-checkpoints/model1/BNN-100000
INFO:tensorflow:Restoring parameters from ./pretrained-checkpoints/model1/BNN-100000
Loading from pre-existing model
On graph 1:

Reading checkpoints...
Loading from checkpoint: BNN-100000
./pretrained-checkpoints/model2/BNN-100000
INFO:tensorflow:Restoring parameters from ./pretrained-checkpoints/model2/BNN-100000
Loading from pre-existing model
On graph 2:

Reading checkpoints...
Loading from checkpoint: BNN-100000
./pretrained-checkpoints/model3/BNN-100000
INFO:tensorflow:Restoring parameters from ./pretrained-checkpoints/model3/BNN-100000
Loading from pre-existing model
On graph 3:

Reading checkpoints...
Loading from checkpoint: BNN-100000
./pretrained-checkpoints/model4/BNN-100000
INFO:tensorflow:Restoring parameters from ./pretrained-checkpoints/model4/BNN-100000
Loading from pre-existing model
On graph 4:

Reading checkpoints...
Loading from checkpoint: B

In [7]:
def predict_ensemble(NNs, X):
    y_pred = []
    y_pred_noise_sq = []
    
    for i in range(num_NNs):
        with graphs[i].as_default():   
            y_pred.append(NNs[i].predict(X))
            y_pred_noise_sq.append(batch_retrieve(NNs[i], NNs[i].noise_sq, X))

    y_preds_train = np.array(y_pred)
    y_preds_noisesq_train = np.array(y_pred_noise_sq)
    y_preds_mu_train = np.mean(y_preds_train, axis=0)
    y_preds_std_train_epi = np.std(y_preds_train, axis=0)
    y_preds_std_train = np.sqrt(np.mean((y_preds_noisesq_train + np.square(y_preds_train)), axis = 0) - np.square(y_preds_mu_train))
    
    return y_preds_train, y_preds_mu_train, y_preds_std_train, y_preds_std_train_epi, y_preds_noisesq_train


In [8]:
y_preds_train, y_pred_train, y_std_train, y_epi_train, y_noise_train = predict_ensemble(NNs, X_train)
y_preds_test, y_pred_test, y_std_test, y_epi_test, y_noise_test = predict_ensemble(NNs, X_test)
y_preds_interp, y_pred_interp, y_std_interp, y_epi_interp, y_noise_interp = predict_ensemble(NNs, X_interp)
y_preds_extrap, y_pred_extrap, y_std_extrap, y_epi_extrap, y_noise_extrap = predict_ensemble(NNs, X_extrap)
y_preds_at, y_pred_at, y_std_at, y_epi_at, y_noise_at = predict_ensemble(NNs, X_at)

In [9]:
print('Training RMSE: {}'.format(calc_RMSE(y_pred_train, y_train)))
print('Testing RMSE: {}'.format(calc_RMSE(y_pred_test, y_test)))
print('Interp RMSE: {}'.format(calc_RMSE(y_pred_interp, y_interp)))
print('Extrap RMSE: {}'.format(calc_RMSE(y_pred_extrap, y_extrap)))

Training RMSE: 0.024788517504930496
Testing RMSE: 0.026396289467811584
Interp RMSE: 0.02757001481950283
Extrap RMSE: 0.02613546885550022


In [10]:
print('Training NLL: {}'.format(calc_NLL(y_pred_train, y_train, y_std_train)))
print('Testing NLL: {}'.format(calc_NLL(y_pred_test, y_test, y_std_test)))
print('Interp NLL: {}'.format(calc_NLL(y_pred_interp, y_interp, y_std_interp)))
print('Extrap NLL: {}'.format(calc_NLL(y_pred_extrap, y_extrap, y_std_extrap)))

Training NLL: -3.465691328048706
Testing NLL: -3.4252800941467285
Interp NLL: -3.1489672660827637
Extrap NLL: -3.494030475616455


In [11]:
print('For train')
report_on_percentiles(y_pred_train, y_train, y_std_train)
print('For test')
report_on_percentiles(y_pred_test, y_test, y_std_test)
print('For Extrap')
report_on_percentiles(y_pred_extrap, y_extrap, y_std_extrap)
print('For Interp')
report_on_percentiles(y_pred_interp, y_interp, y_std_interp)

For train
Using 364995 data points
80.28027781202482 within 1 std
97.7827093521829 within 2 std
99.61232345648571 within 3 std
For test
Using 19286 data points
79.13512392409001 within 1 std
97.43337135746137 within 2 std
99.48148916312351 within 3 std
For Extrap
Using 29986 data points
77.61288601347296 within 1 std
97.41879543787101 within 2 std
99.52978056426332 within 3 std
For Interp
Using 24936 data points
70.97770291947386 within 1 std
94.68639717677254 within 2 std
99.05357715752326 within 3 std


## Save raw BNN output - this data is still logged scaled - to preserve uncertainty

In [12]:
def recube(in_array):

    plev_len = 52
    lat_len = 36
    time_len = 31 * 12

    output = np.zeros([time_len, plev_len, lat_len])

    for t in range(time_len):
        output[t,:,:] = in_array[plev_len * lat_len * (t): plev_len * lat_len * (t+1)].reshape([plev_len, lat_len])
    
    return output

In [13]:
save_dir = './outputRAW'.format(ep_)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
save_coords(data_file, save_dir)

In [14]:
weights = np.mean(get_weights(NNs, X_at), axis=2)
pkl.dump(weights, open(os.path.join(save_dir, 'weights.pkl'), 'wb'))

In [15]:
bias = np.mean(get_bias(NNs, X_at), axis=1)
pkl.dump(recube(bias), open(os.path.join(save_dir, 'bias.pkl'), 'wb'))

In [16]:
pkl.dump(y_pred_at, open(os.path.join(save_dir, 'pred.pkl'), 'wb'))

In [17]:
pkl.dump(y_std_at, open(os.path.join(save_dir, 'std.pkl'), 'wb'))

In [18]:
pkl.dump(recube(y_epi_at), open(os.path.join(save_dir, 'epi.pkl'), 'wb'))

In [19]:
pkl.dump(recube(np.mean(y_noise_at, axis=0)), open(os.path.join(save_dir, 'noise.pkl'), 'wb'))